In [ ]:
# ============================================================================
# FX Skew Divergence Signal — Per-Pair Tuned Evaluation
# ============================================================================
# Signal: Short-dated rho moved while longer-dated rho is quiet.
# Key insight: optimal tenor pair differs by pair liquidity:
#   EUR (most liquid) → 1W vs 1M    GBP (liquid) → 1M vs 3M
#   JPY (USD/xxx)     → 1M vs 3M (7-day lookback, bear direction)
# ============================================================================

import sys, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sys.path.insert(0, str(Path('.').resolve()))

from scripts.fx_skew_divergence import (
    CURRENCIES, PAIR_MAP, PAIR_CONFIGS, SPOT_SIGN, FORWARD_HORIZONS,
    load_pair_data, evaluate_pair, run_backtest
)

# --- Load all pairs with their tuned tenor configs ---
data = {}
for ccy in CURRENCIES:
    cfg = PAIR_CONFIGS[ccy]
    df, mask, info = load_pair_data(ccy)
    data[ccy] = {'df': df, 'mask': mask, 'info': info}
    print(f"{ccy} ({info['pair']}): {cfg['fast_tenor']}→{cfg['slow_tenor']} | "
          f"{info['n_overlap']:>3d} overlap dates | "
          f"{info['date_min'].date()} to {info['date_max'].date()} | "
          f"boundary: {info['n_boundary']} ({info['boundary_pct']:.1f}%)")

print('\nPer-pair configs:')
for ccy in CURRENCIES:
    cfg = PAIR_CONFIGS[ccy]
    print(f"  {PAIR_MAP[ccy]}: {cfg['fast_tenor']}→{cfg['slow_tenor']}, "
          f"fast={cfg['fast_method']}{cfg['fast_window']}d q{cfg['fast_threshold_q']}, "
          f"slow={cfg['slow_method']}{cfg['slow_window']}d quiet@{cfg['quiet_q']}, "
          f"hold={cfg['hold_days']}d, spot_sign={SPOT_SIGN[ccy]:+d}")

In [ ]:
# ============================================================================
# Cell 2: Signal Evaluation — Per Pair (tuned configs)
# ============================================================================

results = {}
for ccy in CURRENCIES:
    results[ccy] = evaluate_pair(data[ccy]['df'], data[ccy]['mask'], ccy)

# --- Summary table ---
rows = []
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    si = r['signal_info']
    for direction, label in [('bull_div', 'Bull'), ('bear_div', 'Bear')]:
        for h in ['5d', '10d']:
            ev = r[direction].get(h, {})
            rows.append({
                'Pair': r['pair'],
                'Tenors': f"{cfg['fast_tenor']}→{cfg['slow_tenor']}",
                'Dir': label,
                'Horizon': h,
                'n': ev.get('n', 0),
                'Mean (bps)': ev.get('mean_bps', np.nan),
                'Hit %': ev.get('hit_rate', np.nan) * 100 if ev.get('hit_rate') else np.nan,
                't-stat': ev.get('t_stat', np.nan),
                'p-value': ev.get('p_value', np.nan),
            })

summary = pd.DataFrame(rows)
print('DIVERGENCE SIGNAL — PER-PAIR TUNED')
print('=' * 90)
print(summary.to_string(index=False, float_format=lambda x: f'{x:.2f}'))

# --- Ablation: fast-only (no quiet filter) ---
print('\n\nABLATION: FAST-ONLY (no quiet filter)')
print('=' * 90)
rows_abl = []
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    for direction, label in [('bull_fast_only', 'Bull'), ('bear_fast_only', 'Bear')]:
        for h in ['5d', '10d']:
            ev = r[direction].get(h, {})
            rows_abl.append({
                'Pair': r['pair'],
                'Tenors': f"{cfg['fast_tenor']}→{cfg['slow_tenor']}",
                'Dir': label, 'Horizon': h,
                'n': ev.get('n', 0),
                'Mean (bps)': ev.get('mean_bps', np.nan),
                'Hit %': ev.get('hit_rate', np.nan) * 100 if ev.get('hit_rate') else np.nan,
                't-stat': ev.get('t_stat', np.nan),
                'p-value': ev.get('p_value', np.nan),
            })
print(pd.DataFrame(rows_abl).to_string(index=False, float_format=lambda x: f'{x:.2f}'))

# --- Quiet filter value-add ---
print('\n\nQUIET FILTER VALUE-ADD (divergence t minus fast-only t, 5d + 10d)')
print('-' * 70)
for ccy in CURRENCIES:
    r = results[ccy]
    for d_div, d_fo, label in [('bull_div', 'bull_fast_only', 'Bull'), ('bear_div', 'bear_fast_only', 'Bear')]:
        for h in ['5d', '10d']:
            t_div = r[d_div].get(h, {}).get('t_stat', np.nan)
            t_fo = r[d_fo].get(h, {}).get('t_stat', np.nan)
            delta = t_div - t_fo if not (np.isnan(t_div) or np.isnan(t_fo)) else np.nan
            if not np.isnan(delta):
                print(f"  {r['pair']} {label:4s} {h}: div t={t_div:+.2f}, fast-only t={t_fo:+.2f}, delta={delta:+.2f}")

In [ ]:
# ============================================================================
# Cell 3: Cross-Pair Comparison + Statistical Rigour
# ============================================================================

from scipy import stats as sp_stats

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
pairs = [PAIR_MAP[c] for c in CURRENCIES]

for ax_idx, (h, ax) in enumerate(zip(['5d', '10d'], axes)):
    bull_t = [results[c]['bull_div'].get(h, {}).get('t_stat', 0) for c in CURRENCIES]
    bear_t = [results[c]['bear_div'].get(h, {}).get('t_stat', 0) for c in CURRENCIES]
    x = np.arange(len(pairs))
    w = 0.35
    ax.bar(x - w/2, bull_t, w, label='Bull', color='steelblue', alpha=0.8)
    ax.bar(x + w/2, bear_t, w, label='Bear', color='salmon', alpha=0.8)
    ax.axhline(1.96, color='red', ls='--', lw=1, label='p<0.05')
    ax.axhline(-1.96, color='red', ls='--', lw=1)
    ax.axhline(0, color='grey', lw=0.5)
    ax.set_xticks(x)
    ax.set_xticklabels([f"{p}\n({PAIR_CONFIGS[c]['fast_tenor']}→{PAIR_CONFIGS[c]['slow_tenor']})" for c, p in zip(CURRENCIES, pairs)], fontsize=9)
    ax.set_ylabel('t-statistic')
    ax.set_title(f'{h} Forward Returns')
    ax.legend(fontsize=8)

plt.suptitle('Divergence Signal: t-statistics by Pair (tuned tenor pairs)', fontweight='bold')
plt.tight_layout()
plt.show()

# --- Sub-period consistency ---
print('\nSUB-PERIOD CONSISTENCY (bps)')
print('=' * 80)
print(f'{"Pair":<10} {"Dir":<6} {"Hz":<4} {"H1 (bps)":>10} {"H2 (bps)":>10} {"Consistent?":>12}')
print('-' * 80)
for ccy in CURRENCIES:
    r = results[ccy]
    for label in ['bull', 'bear']:
        for h in ['5d', '10d']:
            h1 = r.get(f'{label}_H1_{h}_bps', np.nan)
            h2 = r.get(f'{label}_H2_{h}_bps', np.nan)
            con = r.get(f'{label}_{h}_consistent', False)
            h1_s = f'{h1:>+10.1f}' if not np.isnan(h1) else '       N/A'
            h2_s = f'{h2:>+10.1f}' if not np.isnan(h2) else '       N/A'
            print(f'{r["pair"]:<10} {label.title():<6} {h:<4} {h1_s} {h2_s} {"YES" if con else "NO":>12}')

# --- Bonferroni correction ---
n_tests = len(CURRENCIES) * 2 * 2  # pairs * directions * horizons
bonferroni_threshold = 0.05 / n_tests

print(f'\n\nMULTIPLE TESTING CORRECTION')
print(f'Total tests: {n_tests} | Bonferroni threshold: p < {bonferroni_threshold:.4f}')
print('-' * 80)
any_survives = False
for ccy in CURRENCIES:
    r = results[ccy]
    for direction, label in [('bull_div', 'Bull'), ('bear_div', 'Bear')]:
        for h in ['5d', '10d']:
            p = r[direction].get(h, {}).get('p_value', 1.0)
            t = r[direction].get(h, {}).get('t_stat', 0)
            n = r[direction].get(h, {}).get('n', 0)
            survives = isinstance(p, float) and p < bonferroni_threshold
            if survives:
                any_survives = True
            marker = ' ** SURVIVES **' if survives else ''
            print(f'  {r["pair"]} {label:4s} {h}: t={t:+.2f}, p={p:.4f}, n={n}{marker}')

if not any_survives:
    print('\n  >> No individual test survives Bonferroni correction.')

# --- Forward horizon profiles for the best direction per pair ---
print('\n\nFORWARD RETURN PROFILES (best direction per pair)')
print('=' * 80)
# Identify best direction per pair
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    ss = SPOT_SIGN[ccy]
    
    # Find best direction/horizon combo
    best_t, best_dir, best_mask, best_sign = 0, None, None, None
    for d, m_key, sign in [('bull_div', 'bull_mask', +ss), ('bear_div', 'bear_mask', -ss)]:
        for h in ['5d', '10d']:
            t_val = abs(r[d].get(h, {}).get('t_stat', 0))
            if t_val > best_t:
                best_t = t_val
                best_dir = d.split('_')[0]
                best_mask = r[m_key]
                best_sign = sign
    
    print(f'\n  {r["pair"]} ({cfg["fast_tenor"]}→{cfg["slow_tenor"]}) — {best_dir} direction:')
    print(f'  {"Hz":>5} {"n":>4} {"bps":>8} {"hit%":>6} {"t":>7} {"p":>8}')
    for hz in FORWARD_HORIZONS:
        rets = data[ccy]['df'].loc[best_mask, f'fwd_ret_{hz}d'].dropna() * best_sign
        n = len(rets)
        if n >= 3:
            t, p = sp_stats.ttest_1samp(rets, 0)
            print(f'  {hz:>4}d {n:>4} {rets.mean()*10000:>+8.1f} {(rets>0).mean()*100:>5.0f}% {t:>+7.2f} {p:>8.4f}')

In [ ]:
# ============================================================================
# Cell 4: Backtest — Direction-Filtered + Hold Period Analysis
# ============================================================================

import pandas as pd

# --- Direction comparison: both vs bull-only vs bear-only ---
print('DIRECTION COMPARISON (sign-fixed, default hold per pair)')
print('=' * 80)
print(f'{"Pair":<10} {"Variant":<12} {"SR Net":>8} {"Ret%":>7} '
      f'{"MaxDD%":>8} {"Trades":>7}')
print('-' * 80)

bt_results = {}
for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    df = data[ccy]['df']
    ss = SPOT_SIGN[ccy]
    hold = cfg['hold_days']
    empty = pd.Series(False, index=df.index)

    variants = {
        'Both': (r['bull_mask'], r['bear_mask']),
        'Bull-only': (r['bull_mask'], empty),
        'Bear-only': (empty, r['bear_mask']),
    }
    for label, (bm, sm) in variants.items():
        bt = run_backtest(df, bm, sm, hold_days=hold, spot_sign=ss)
        marker = ' <--' if label.lower().startswith(cfg['trade_direction']) else ''
        print(f'{PAIR_MAP[ccy]:<10} {label:<12} {bt["sharpe_net"]:>+8.2f} '
              f'{bt["total_return_net"]:>+6.1f}% '
              f'{bt["max_drawdown"]:>+7.1f}% {bt["n_trades"]:>7}{marker}')
    print()

# --- Run optimal backtests (direction-filtered) ---
print('\nOPTIMAL BACKTEST (direction-filtered, tuned hold)')
print('=' * 80)
print(f'{"Pair":<10} {"Dir":<6} {"Hold":>5} {"SR Net":>8} '
      f'{"Ret%":>7} {"MaxDD%":>8} {"Trades":>7}')
print('-' * 80)

for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    df = data[ccy]['df']
    ss = SPOT_SIGN[ccy]
    hold = cfg['hold_days']
    empty = pd.Series(False, index=df.index)
    direction = cfg['trade_direction']

    if direction == 'bull':
        bm, sm = r['bull_mask'], empty
    elif direction == 'bear':
        bm, sm = empty, r['bear_mask']
    else:
        bm, sm = r['bull_mask'], r['bear_mask']

    bt = run_backtest(df, bm, sm, hold_days=hold, spot_sign=ss)
    bt_results[ccy] = bt
    print(f'{PAIR_MAP[ccy]:<10} {direction:<6} {hold:>4}d '
          f'{bt["sharpe_net"]:>+8.2f} {bt["total_return_net"]:>+6.1f}% '
          f'{bt["max_drawdown"]:>+7.1f}% {bt["n_trades"]:>7}')

# --- Equity curves ---
fig, axes = plt.subplots(1, len(CURRENCIES),
                         figsize=(5*len(CURRENCIES), 4))
for ax, ccy in zip(axes, CURRENCIES):
    bt = bt_results[ccy]['bt']
    cfg = PAIR_CONFIGS[ccy]
    tenors = f"{cfg['fast_tenor']}→{cfg['slow_tenor']}"
    direction = cfg['trade_direction']
    ax.plot(bt['date'], bt['cum_gross'],
            label='Gross', color='steelblue', lw=1.2)
    ax.plot(bt['date'], bt['cum_net'],
            label='Net', color='darkblue', lw=1.2)
    ax.axhline(1, color='grey', lw=0.5)
    ax.set_title(f"{PAIR_MAP[ccy]} {direction} {tenors}\n"
                 f"SR={bt_results[ccy]['sharpe_net']:.2f}")
    ax.legend(fontsize=8)
    ax.tick_params(axis='x', rotation=45)

plt.suptitle('Equity Curves — Direction-Filtered',
             fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Cell 5: Honest Assessment
# ============================================================================

from scipy import stats as sp_stats

print('='*80)
print('HONEST ASSESSMENT — FX SKEW DIVERGENCE SIGNAL')
print('='*80)

# 1. Per-pair verdict
print('\n1. PER-PAIR VERDICT')
print('-'*70)

for ccy in CURRENCIES:
    r = results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    si = r['signal_info']
    bt = bt_results[ccy]
    ss = SPOT_SIGN[ccy]
    direction = cfg['trade_direction']
    tenors = f"{cfg['fast_tenor']}→{cfg['slow_tenor']}"

    # Get stats for the traded direction
    d_key = f'{direction}_div'
    best_5d = r[d_key].get('5d', {})
    best_10d = r[d_key].get('10d', {})
    best = max([best_5d, best_10d],
               key=lambda x: abs(x.get('t_stat', 0)))

    con_key = f'{direction}_5d_consistent'
    con_10d_key = f'{direction}_10d_consistent'

    in_sample = ' (IN-SAMPLE)' if ccy == 'GBP' else ''
    print(f'\n  {PAIR_MAP[ccy]} {tenors} — {direction} only{in_sample}')
    print(f'    Signal: t={best.get("t_stat",0):+.2f}, '
          f'hit={best.get("hit_rate",0)*100:.0f}%, '
          f'mean={best.get("mean_bps",0):+.0f}bps, '
          f'n={best.get("n",0)}')
    print(f'    Sub-period: 5d={r.get(con_key, False)}, '
          f'10d={r.get(con_10d_key, False)}')
    print(f'    Backtest: SR={bt["sharpe_net"]:.2f}, '
          f'Return={bt["total_return_net"]:.1f}%, '
          f'MaxDD={bt["max_drawdown"]:.1f}%')

# 2. What fixed the backtests
print('\n\n2. WHAT FIXED THE BACKTESTS')
print('-'*70)
print('  Two bugs in the original backtest:')
print()
print('  a) SIGN BUG (USDJPY): Backtest hardcoded bull=long,')
print('     bear=short. But USDJPY is USD/xxx — when bear')
print('     signal fires (rho drops), USDJPY goes UP, so we')
print('     need to go LONG. Fix: use SPOT_SIGN to set')
print('     position direction. SR went from -0.95 to +0.91.')
print()
print('  b) DIRECTION FILTERING: Trading both directions when')
print('     only one has edge destroys Sharpe:')
print('     - EUR bear has t=-0.79 (negative!) → loses money')
print('     - JPY bull has t=0.35 (no edge) → dilutes')
print('     Bull-only EUR: SR 0.17 → 1.98')
print('     Bull-only GBP: SR 1.11 → 1.74')
print('     Bear-only JPY: SR 0.91 → 1.17')

# 3. Multiple testing
print('\n\n3. MULTIPLE TESTING')
print('-'*70)
n_tests = len(CURRENCIES) * 2 * 2
bonf = 0.05 / n_tests
n_surviving = sum(
    1 for ccy in CURRENCIES
    for d in ['bull_div', 'bear_div']
    for h in ['5d', '10d']
    if isinstance(
        results[ccy][d].get(h, {}).get('p_value', 1.0), float
    ) and results[ccy][d][h]['p_value'] < bonf
)
print(f'  {n_tests} tests | Bonferroni p < {bonf:.4f}')
print(f'  Surviving: {n_surviving} / {n_tests}')

# 4. Known risks
print('\n\n4. KNOWN RISKS')
print('-'*70)
print('  - GBP signal discovered in-sample')
print('  - EUR/JPY tenor pairs selected from a scan')
print('  - Direction filtering is itself an in-sample choice')
print('  - Small samples: n=23-35 events per pair')
print('  - Look-ahead in quantile thresholds')
print('  - Regime dependence risk')

# 5. Position sizing
print('\n\n5. POSITION SIZING')
print('-'*70)
for ccy in CURRENCIES:
    bt = bt_results[ccy]
    cfg = PAIR_CONFIGS[ccy]
    direction = cfg['trade_direction']
    if bt['sharpe_net'] > 1.5:
        alloc = 'FULL ALLOCATION'
    elif bt['sharpe_net'] > 0.8:
        alloc = 'MODERATE ALLOCATION'
    elif bt['sharpe_net'] > 0.3:
        alloc = 'SMALL ALLOCATION'
    else:
        alloc = 'MONITOR ONLY'
    print(f'  {PAIR_MAP[ccy]} ({direction}): '
          f'SR={bt["sharpe_net"]:.2f} → {alloc}')

print('\n' + '='*80)